# Data Preprocessing

This notebook cleans, tokenizes and augments data

In [1]:
import pandas as pd
import re
import os
from dotenv import load_dotenv
from tqdm import tqdm
import random
import numpy as np
import warnings

# Tokenization
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

# Data Augmentation
import openai

# Burrow's Delta Score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cityblock  # Burrows' Delta uses Manhattan distance


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hadrienstrichard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/hadrienstrichard/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Cleaning and tokenizing

In [2]:
# Load initial data

texts = pd.read_parquet('../Data/excerpts_df.parquet')
final_authors = pd.read_csv('../Data/final_authors.csv')

In [3]:
# Basic cleaning function tailored for stylometry (preserving punctuation, no lemmatization)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # remove emails
    text = re.sub(r'\d+', '', text)  # remove digits
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    text = re.sub(r'[“”«»]', '"', text)  # normalize quotes
    text = re.sub(r"[’]", "'", text)  # normalize apostrophes
    return text.strip()

In [4]:
# Apply cleaning
texts['Cleaned_Text'] = texts['Excerpt_Text'].apply(clean_text)

In [5]:
# Display first 3 cleaned text for verification
pd.set_option('display.max_colwidth', None)
print(texts['Cleaned_Text'][:3])

0                                                                              n'est souvent déterminée que par un mot. en ce point même, cependant, il n'a fait que se conformer au caprice piquant de la nature, qui se joue à nous faire parcourir dans la durée d'un seul rêve, plusieurs fois interrompu par des épisodes étrangers à son objet, tous les développements d'une action régulière, complète et plus ou moins vraisemblable. les personnes qui ont lu apulée s'apercevront facilement que la fable du premier livre de l'_âne d'or_ de cet ingénieux conteur a beaucoup de rapports avec celle-ci, et qu'elles se ressemblent par le fond presque autant qu'elles diffèrent par la forme. l'auteur paraît même avoir affecté de solliciter ce rapprochement en conservant à son principal personnage le nom de lucius. le récit du philosophe de madaure et celui du prêtre dalmate, cité par fortis, tome i, page , ont en effet une origine commune dans les chants traditionnels d'une contrée qu'apulée avait curi

In [6]:
# Load French tokenizer
punkt_param = PunktParameters()
tokenizer = PunktSentenceTokenizer(punkt_param)

In [7]:
# Apply word tokenization
texts['Tokens'] = texts['Cleaned_Text'].apply(lambda x: word_tokenize(x, language='french'))

In [8]:
# Display first 3 tokenized text for verification
print(texts['Tokens'][:3])

0                                                                     [n'est, souvent, déterminée, que, par, un, mot, ., en, ce, point, même, ,, cependant, ,, il, n, ', a, fait, que, se, conformer, au, caprice, piquant, de, la, nature, ,, qui, se, joue, à, nous, faire, parcourir, dans, la, durée, d'un, seul, rêve, ,, plusieurs, fois, interrompu, par, des, épisodes, étrangers, à, son, objet, ,, tous, les, développements, d'une, action, régulière, ,, complète, et, plus, ou, moins, vraisemblable, ., les, personnes, qui, ont, lu, apulée, s'apercevront, facilement, que, la, fable, du, premier, livre, de, l'_âne, d'or_, de, cet, ingénieux, conteur, a, beaucoup, de, rapports, avec, celle-ci, ,, et, qu'elles, se, ...]
1                                                                                                              [le, reste, ne, me, regarde, point, ., j'ai, dit, de, qui, était, la, fable, :, sauf, quelques, phrases, de, transition, ,, tout, appartient, à, homère, ,, à, théocrite,

In [9]:
# Drop authors and excerpts with < 25 excerpts
author_counts = texts['Author'].value_counts()
valid_authors = author_counts[author_counts > 25].index
texts = texts[texts['Author'].isin(valid_authors)]

# Filter final_authors
final_authors = final_authors[final_authors['Name'].isin(valid_authors)]

# Update books.csv
books = pd.read_csv('../Data/books.csv')
books = books[books['Author'].isin(valid_authors)]
books.to_csv('../Data/books.csv', index=False)

# Save updated data
texts.to_parquet('../Data/excerpts_df.parquet')
final_authors.to_csv('../Data/final_authors.csv', index=False)

In [10]:
final_authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        28 non-null     object
 1   birth_date  28 non-null     object
 2   death_date  28 non-null     object
 3   n_books     28 non-null     int64 
 4   Mouvement   28 non-null     object
 5   Genres      28 non-null     object
 6   n_excerpts  28 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 1.7+ KB


In [11]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Author     299 non-null    object 
 1   Title      299 non-null    object 
 2   Language   299 non-null    object 
 3   EBook-No   299 non-null    int64  
 4   URL        299 non-null    object 
 5   Roman      299 non-null    bool   
 6   Genres     299 non-null    object 
 7   Mouvement  299 non-null    object 
 8   Date       298 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 19.1+ KB


In [12]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14680 entries, 0 to 14679
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Author              14680 non-null  object
 1   Title               14680 non-null  object
 2   URL                 14680 non-null  object
 3   Excerpt_ID          14680 non-null  object
 4   Excerpt_Text        14680 non-null  object
 5   Cleaned_Text        14680 non-null  object
 6   Tokens              14680 non-null  object
 7   Tokenized_Text      55 non-null     object
 8   of_which_generated  55 non-null     object
dtypes: object(9)
memory usage: 1.0+ MB


## Data Aug

In [13]:
# Load OpenAI key

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [14]:
# Delta Score Calculation

def compute_author_profiles(texts_df):
    """Creates stylometric profiles per author"""
    vectorizer = CountVectorizer(max_features=300, analyzer='word', lowercase=True)
    X = vectorizer.fit_transform(texts_df["Cleaned_Text"])
    features = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    features["Author"] = texts_df["Author"].values

    # Compute mean frequencies per author
    author_profiles = features.groupby("Author").mean()

    # Z-score normalization (feature-wise)
    scaler = StandardScaler()
    scaled_profiles = pd.DataFrame(
        scaler.fit_transform(author_profiles),
        index=author_profiles.index,
        columns=author_profiles.columns
    )
    return scaled_profiles, scaler, vectorizer

def delta_score(text, scaled_author_profiles, scaler, vectorizer):
    """Compute Delta between one excerpt and all author profiles"""
    vec = vectorizer.transform([text])
    vec_df = pd.DataFrame(vec.toarray(), columns=vectorizer.get_feature_names_out())

    # Align columns
    for col in scaled_author_profiles.columns:
        if col not in vec_df.columns:
            vec_df[col] = 0
    vec_df = vec_df[scaled_author_profiles.columns]

    # Normalize using same scaler as author profiles
    vec_scaled = pd.DataFrame(
        scaler.transform(vec_df),
        columns=vec_df.columns
    ).iloc[0]

    # Compute Manhattan (L1) distance: Burrows’ Delta
    deltas = {}
    for author in scaled_author_profiles.index:
        profile = scaled_author_profiles.loc[author]
        deltas[author] = cityblock(vec_scaled, profile)

    return deltas


def get_delta_distribution_for_author(author, texts_df, profiles, scaler, vectorizer):
    subset = texts_df[texts_df["Author"] == author]["Cleaned_Text"]
    deltas = [delta_score(text, profiles, scaler, vectorizer)[author] for text in subset]
    return deltas

In [15]:
def build_prompt(target_author, example_1, example_2, input_excerpt, author_metadata):
    bio = (
        f"{target_author} est un écrivain français du {author_metadata['birth_date'][:4]}–{author_metadata['death_date'][:4]} siècle. "
        f"Appartenant au mouvement {author_metadata['Mouvement']}, il a principalement écrit dans les genres suivants : "
        f"{author_metadata['Genres']}."
    )

    return f"""
Tu es un modèle de génération de texte littéraire spécialisé dans l'augmentation de données pour la classification d'auteurs.

Ton rôle :
- Générer des extraits de fiction de 1500 mots (+/- 10%) en français.
- Respecter scrupuleusement le style de l’auteur : vocabulaire, syntaxe, ponctuation, ton, époque.
- Ne pas inclure de dialogues, titres ou en-têtes : que du texte narratif ou descriptif.
- Générer un passage qui suit naturellement celui donné, comme s’il s’agissait de la suite immédiate dans le livre.

Informations :
{bio}

Exemple de suite entre deux extraits d’un même livre de cet auteur:
--- Extrait A ---
{example_1}
--- Extrait B (suite de A) ---
{example_2}

Tâche :
Voici un extrait tiré de l’œuvre suivante : "{input_excerpt['Title']}". Génére la suite immédiate de cet extrait dans le même style, sans introduire de coupure ou de résumé.

--- Extrait donné ---
{input_excerpt['Excerpt_Text']}
--- Ta réponse (suite de l'extrait) ---
"""


In [16]:
def generate_and_validate(author_row, texts_df, n_needed, author_profiles, vectorizer, scaler, client):

    author_name = author_row["Name"]
    subset = texts_df[texts_df["Author"] == author_name].copy()

    # Extract book ID and position
    subset[["Book_ID", "Book_Pos"]] = subset["Excerpt_ID"].str.extract(r"(\d+)_(\d+)")
    subset["Book_Pos"] = subset["Book_Pos"].astype(int)

    # For each book, keep middle 50% of excerpts
    middle_excerpts = []
    for book_id, group in subset.groupby("Book_ID"):
        group_sorted = group.sort_values("Book_Pos")
        n = len(group_sorted)
        if n >= 4:
            middle = group_sorted.iloc[n // 4: 3 * n // 4]
            middle_excerpts.append(middle)
        else:
            # If too few excerpts, keep them all (fallback)
            middle_excerpts.append(group_sorted)

    middle_subset = pd.concat(middle_excerpts).reset_index(drop=True)


    new_excerpts = []
    attempts = 0
    max_attempts = 5 * n_needed 

    while len(new_excerpts) < n_needed and attempts < max_attempts:
        attempts += 1

        # Random consecutive excerpts for example
        examples = middle_subset.sample(2).sort_values("Excerpt_ID")
        example_1, example_2 = examples.iloc[0]["Excerpt_Text"], examples.iloc[1]["Excerpt_Text"]

        # Input excerpt also from middle
        input_excerpt = middle_subset.sample(1).iloc[0]

        prompt = build_prompt(
            target_author=author_name,
            example_1=example_1,
            example_2=example_2,
            input_excerpt=input_excerpt,
            author_metadata=author_row
        )

        try:
            print(f"Generating excerpt for {author_name}... (Attempt {attempts})")
            response = client.chat.completions.create(
                model="gpt-4.1-2025-04-14",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
                max_tokens=2048
            )
            generated = response.choices[0].message.content
            print(f"Generated excerpt for {author_name}: {generated[:100]}...")

            cleaned_gen = clean_text(generated)
            tokenized_gen = word_tokenize(cleaned_gen, language='french')
            print(len(tokenized_gen), "tokens in generated excerpt")
            length_ok = 1350 <= len(tokenized_gen) <= 1650

            delta_scores = delta_score(cleaned_gen, author_profiles, scaler, vectorizer)
            print(f"Delta scores for {author_name}: {delta_scores}")
            sorted_authors = sorted(delta_scores.items(), key=lambda x: x[1])
            closest_author, closest_score = sorted_authors[0]
            target_score = delta_scores[author_name]
            print(f"Closest author: {closest_author} ({closest_score:.2f}), Target author ({author_name}): {target_score:.2f}")

            # Allow small tolerance if author is nearly the closest
            tolerance = attempts / 100  # Make the tolerance increase with attempts
            delta_gap = abs(closest_score - target_score)
            is_closest_or_near = (
                closest_author == author_name or
                (target_score - closest_score) / closest_score <= tolerance
            )

            # Delta range check
            real_deltas = get_delta_distribution_for_author(
                author_name, texts_df, author_profiles, scaler, vectorizer
            )
            threshold = np.percentile(real_deltas, 90)
            within_range = target_score <= threshold

            print(f"Delta to {author_name}: {target_score:.2f} | 90th percentile: {threshold:.2f}")
            good_style = is_closest_or_near and within_range
            print(f"Excerpt length OK: {length_ok}, Good style: {good_style}")

            if length_ok and good_style:
                new_excerpts.append({
                    "Author": author_name,
                    "Title": input_excerpt["Title"],
                    "URL": input_excerpt["URL"],
                    "Excerpt_ID": f"{input_excerpt['Excerpt_ID']}_gen{len(new_excerpts)}",
                    "Excerpt_Text": generated,
                    "Cleaned_Text": cleaned_gen,
                    "Tokens": tokenized_gen,
                    "of_which_generated": input_excerpt["Excerpt_ID"]
                })
                print(f"✅ Accepted excerpt for {author_name}: {len(new_excerpts)} / {n_needed}")

        except Exception as e:
            print(f"Error (attempt {attempts}):", e)
            continue

    if len(new_excerpts) < n_needed:
        warnings.warn(
            f"⚠️ Could not generate all excerpts for {author_name}. Only {len(new_excerpts)} out of {n_needed} were accepted. Review needed."
        )

    return pd.DataFrame(new_excerpts)


In [17]:
# Precompute author profiles
author_profiles, scaler, vectorizer = compute_author_profiles(texts)

# Get underrepresented authors
need_aug = final_authors[final_authors["n_excerpts"] < 50]

print(f"Authors needing augmentation: {len(need_aug)}")

Authors needing augmentation: 0


In [18]:
augmented_all = []

client = client = openai.OpenAI()

for _, author_row in tqdm(need_aug.iterrows(), total=need_aug.shape[0]):
    n_needed = 50 - author_row["n_excerpts"]
    df_aug = generate_and_validate(author_row, texts, n_needed, author_profiles, vectorizer, scaler, client)
    augmented_all.append(df_aug)

augmented_df = pd.concat(augmented_all, ignore_index=True)

# Add new excerpts
texts = pd.concat([texts, augmented_df], ignore_index=True)

# Update author excerpt count
updated_counts = texts["Author"].value_counts().reset_index()
updated_counts.columns = ["Name", "n_excerpts"]
final_authors.update(updated_counts)

# Fill missing "of_which_generated" for real texts
texts["of_which_generated"] = texts.get("of_which_generated", np.nan)

0it [00:00, ?it/s]


ValueError: No objects to concatenate

In [ ]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14680 entries, 0 to 14679
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Author              14680 non-null  object
 1   Title               14680 non-null  object
 2   URL                 14680 non-null  object
 3   Excerpt_ID          14680 non-null  object
 4   Excerpt_Text        14680 non-null  object
 5   Cleaned_Text        14680 non-null  object
 6   Tokens              14625 non-null  object
 7   Tokenized_Text      55 non-null     object
 8   of_which_generated  55 non-null     object
dtypes: object(9)
memory usage: 1.0+ MB


In [ ]:
texts['of_which_generated'].describe()

count           55
unique          38
top       16239_18
freq             3
Name: of_which_generated, dtype: object

In [ ]:
# Save final versions
texts.to_parquet('../Data/excerpts_df.parquet', index=False)
final_authors.to_csv('../Data/final_authors.csv', index=False)
